In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope

In [ ]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
height = 512
width = 512

In [ ]:
dataset_path="/content/drive/MyDrive/Dataset/people_segmentation"

In [ ]:
model_path = os.path.join(dataset_path,"files" ,"unet.h5")
input_video_path = os.path.join(dataset_path,"test_videos", "inputs", "Video-2.mp4")
output_video_path = os.path.join(dataset_path,"test_videos", "outputs", "Video-2.avi")

In [ ]:
def dice_loss(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    dice = (2. * intersection + 1e-15) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1e-15)
    return 1.0 - dice

In [ ]:
with CustomObjectScope({'dice_loss': dice_loss}):
    model = tf.keras.models.load_model(model_path)

In [ ]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 518, 518, 3)          0         ['input_5[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 256, 256, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 256, 256, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                           

In [ ]:
vs = cv2.VideoCapture(input_video_path)
_, frame = vs.read()
H, W, _ = frame.shape
vs.release()

print(H, W)

2160 3840


In [ ]:
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
fps = 30
out = cv2.VideoWriter(output_video_path, fourcc, fps, (W, H), True)

In [ ]:
cap = cv2.VideoCapture(input_video_path)

while True:
    ret, frame = cap.read()
    if ret == False:
        cap.release()
        out.release()
        break

    H, W, _ = frame.shape
    ori_frame = frame

    frame = cv2.resize(frame, (width, height))
    frame = frame/255
    frame = np.expand_dims(frame, axis=0)


    mask = model.predict(frame)[0]
    mask = cv2.resize(mask, (W, H))
    mask = mask > 0.5
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1) * 255

    mask = mask.astype(np.float32)
    ori_frame = ori_frame.astype(np.float32)

    alpha = 0.6
    output = cv2.addWeighted(mask, alpha, ori_frame, 1-alpha, 0)
    output = output.astype(np.uint8)

    out.write(output)

print("Process Completed!")

1/1 [==============================] - 3s 3s/step
Process Completed!


In [ ]:
print(output)

[[[43 48 50]
  [48 53 54]
  [61 66 67]
  ...
  [39 42 46]
  [40 43 47]
  [41 44 48]]

 [[46 52 53]
  [49 54 55]
  [64 69 70]
  ...
  [37 40 44]
  [39 42 46]
  [40 43 46]]

 [[52 57 58]
  [52 58 59]
  [68 73 74]
  ...
  [34 37 41]
  [36 40 43]
  [38 41 45]]

 ...

 [[64 66 66]
  [65 67 66]
  [66 68 67]
  ...
  [52 49 52]
  [52 49 52]
  [52 49 52]]

 [[63 65 64]
  [63 65 64]
  [64 66 65]
  ...
  [52 49 52]
  [52 49 52]
  [52 49 52]]

 [[62 64 63]
  [63 65 64]
  [64 66 65]
  ...
  [52 49 52]
  [52 49 52]
  [52 49 52]]]
